In [3]:
import sys
import pathlib
utils_path = pathlib.Path().absolute().parent.parent
sys.path.append(utils_path.__str__())

import utils.layout as lay
from utils.config import apikey
from eod import EodHistoricalData
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio

In [4]:
pio.templates.default = 'simple_white+blog_mra'

In [5]:
def format_value(val):
    if not pd.isna(val) or val  != "nan":
        return f'{val / 1e6:.2f}'

In [6]:
client = EodHistoricalData(apikey)

In [7]:
stock = "TSLA.US"
peers = ["GM.US", "GOOGL.US", "AMZN.US", "TCEHY.US", "INTC.US", "AAPL.US", "F.US"]
all_ticks = [stock] + peers

df_fund = pd.DataFrame()
for tick in all_ticks:
    print(tick)
    inc_stat = client.get_fundamental_equity(tick, filter_='Financials::Income_Statement::yearly') # Stock
    df_temp = pd.DataFrame.from_dict(inc_stat, orient="index").sort_index()
    df_temp.index = pd.to_datetime(df_temp.index).year.astype(str)
    df_temp["ticker"] = tick
    df_temp = df_temp.set_index([df_temp["ticker"], df_temp.index])
    df_fund= pd.concat([df_fund, df_temp],axis = 0)

df_fund = df_fund.replace(to_replace={None: np.nan, type(None):  np.nan}, inplace=False)
df_fund.index.names = ["ticker", "year"]

TSLA.US
GM.US
GOOGL.US
AMZN.US
TCEHY.US
INTC.US
AAPL.US
F.US


In [8]:
df_fund = df_fund[["totalRevenue", "netIncome", "ebitda"]].astype(float)
df_fund["revGrowth"] = df_fund.groupby("ticker")["totalRevenue"].pct_change()
df_fund["ebitdaMargin"] = df_fund["ebitda"] / df_fund["totalRevenue"]
df_fund["profitMargin"] = df_fund["netIncome"] / df_fund["totalRevenue"]
df_fund = df_fund[df_fund.index.get_level_values(1) != "2023"] ## adjust for fiscal year
df_last = df_fund.groupby(level=['ticker']).tail(3)
df_last

totalRevenue     netIncome        ebitda  revGrowth  \
ticker   year                                                        
TSLA.US  2020  3.153600e+10  6.900000e+08  4.224000e+09   0.283099   
         2021  5.382300e+10  5.519000e+09  9.598000e+09   0.706716   
         2022  8.146200e+10  1.258300e+10  1.783300e+10   0.513517   
GM.US    2020  1.224850e+11  6.427000e+09  2.174300e+10  -0.107493   
         2021  1.270040e+11  1.001900e+10  2.571700e+10   0.036894   
         2022  1.567350e+11  9.934000e+09  2.387400e+10   0.234095   
GOOGL.US 2020  1.825270e+11  4.026900e+10  5.492100e+10   0.127705   
         2021  2.576370e+11  7.603300e+10  9.115500e+10   0.411501   
         2022  2.828360e+11  5.997200e+10  9.077000e+10   0.097808   
AMZN.US  2020  3.860640e+11  2.133100e+10  4.807900e+10   0.376234   
         2021  4.698220e+11  3.336400e+10  5.931200e+10   0.216954   
         2022  5.139830e+11 -2.722000e+09  5.416900e+10   0.093995   
TCEHY.US 2020  4.820640e+11  1.598470e+11  1.841880e+11   0.277705   
         2021  5.601180e+11  2.248220e+11  1.662470e+11   0.161916   
         2022  5.545520e+11  1.882430e+11  1.491310e+11  -0.009937   
INTC.US  2020  7.786700e+10  2.089900e+10  3.801900e+10   0.082012   
         2021  7.902400e+10  1.986800e+10  3.650000e+10   0.014859   
         2022  6.305400e+10  8.014000e+09  1.110500e+10  -0.202091   
AAPL.US  2020  2.745150e+11  5.741100e+10  7.734400e+10   0.055121   
         2021  3.658170e+11  9.468000e+10  1.202330e+11   0.332594   
         2022  3.943280e+11  9.980300e+10  1.305410e+11   0.077938   
F.US     2020  1.271440e+11 -1.276000e+09  1.627500e+10  -0.184452   
         2021  1.363410e+11  1.793700e+10  2.333800e+10   0.072335   
         2022  1.580570e+11 -2.152000e+09  1.889400e+10   0.159277   

               ebitdaMargin  profitMargin  
ticker   year                              
TSLA.US  2020      0.133942      0.021880  
         2021      0.178325      0.102540  
         2022      0.218912      0.154465  
GM.US    2020      0.177516      0.052472  
         2021      0.202490      0.078887  
         2022      0.152321      0.063381  
GOOGL.US 2020      0.300892      0.220619  
         2021      0.353812      0.295117  
         2022      0.320928      0.212038  
AMZN.US  2020      0.124536      0.055252  
         2021      0.126244      0.071014  
         2022      0.105391     -0.005296  
TCEHY.US 2020      0.382082      0.331589  
         2021      0.296807      0.401383  
         2022      0.268922      0.339451  
INTC.US  2020      0.488256      0.268394  
         2021      0.461885      0.251417  
         2022      0.176119      0.127097  
AAPL.US  2020      0.281748      0.209136  
         2021      0.328670      0.258818  
         2022      0.331047      0.253096  
F.US     2020      0.128004     -0.010036  
         2021      0.171174      0.131560  
         2022      0.119539     -0.013615

In [9]:
df_data = pd.DataFrame(index =all_ticks)
marketCap = np.zeros(len(all_ticks))
enterpriseValue = np.zeros(len(all_ticks))

c = 0
for tick in all_ticks:
    print(tick)
    marketCap[c] =  client.get_fundamental_equity(tick, filter_='Highlights::MarketCapitalization')
    enterpriseValue[c] = client.get_fundamental_equity(tick, filter_='Valuation::EnterpriseValue')
    c+=1
    
df_data["marketCap"] = marketCap
df_data["enterpriseValue"] = enterpriseValue
df_data

TSLA.US
GM.US
GOOGL.US
AMZN.US
TCEHY.US
INTC.US
AAPL.US
F.US


,marketCap,enterpriseValue
TSLA.US,7.761015e+11,7.582119e+11
GM.US,3.956428e+10,1.235623e+11
GOOGL.US,1.697607e+12,1.606718e+12
AMZN.US,1.519408e+12,1.592233e+12
TCEHY.US,3.888186e+11,3.912218e+11
INTC.US,1.894670e+11,2.103227e+11
AAPL.US,2.945234e+12,2.994758e+12
F.US,4.239124e+10,1.458183e+11


In [10]:
df_all = pd.merge(df_data, df_last.reset_index(level=1, drop=False), left_index=True, right_index=True)
df_all = df_all.set_index([df_all.index, "year"])
df_all.index.names = ["ticker", "year"]
df_all

marketCap  enterpriseValue  totalRevenue     netIncome  \
ticker   year                                                              
AAPL.US  2020  2.945234e+12     2.994758e+12  2.745150e+11  5.741100e+10   
         2021  2.945234e+12     2.994758e+12  3.658170e+11  9.468000e+10   
         2022  2.945234e+12     2.994758e+12  3.943280e+11  9.980300e+10   
AMZN.US  2020  1.519408e+12     1.592233e+12  3.860640e+11  2.133100e+10   
         2021  1.519408e+12     1.592233e+12  4.698220e+11  3.336400e+10   
         2022  1.519408e+12     1.592233e+12  5.139830e+11 -2.722000e+09   
F.US     2020  4.239124e+10     1.458183e+11  1.271440e+11 -1.276000e+09   
         2021  4.239124e+10     1.458183e+11  1.363410e+11  1.793700e+10   
         2022  4.239124e+10     1.458183e+11  1.580570e+11 -2.152000e+09   
GM.US    2020  3.956428e+10     1.235623e+11  1.224850e+11  6.427000e+09   
         2021  3.956428e+10     1.235623e+11  1.270040e+11  1.001900e+10   
         2022  3.956428e+10     1.235623e+11  1.567350e+11  9.934000e+09   
GOOGL.US 2020  1.697607e+12     1.606718e+12  1.825270e+11  4.026900e+10   
         2021  1.697607e+12     1.606718e+12  2.576370e+11  7.603300e+10   
         2022  1.697607e+12     1.606718e+12  2.828360e+11  5.997200e+10   
INTC.US  2020  1.894670e+11     2.103227e+11  7.786700e+10  2.089900e+10   
         2021  1.894670e+11     2.103227e+11  7.902400e+10  1.986800e+10   
         2022  1.894670e+11     2.103227e+11  6.305400e+10  8.014000e+09   
TCEHY.US 2020  3.888186e+11     3.912218e+11  4.820640e+11  1.598470e+11   
         2021  3.888186e+11     3.912218e+11  5.601180e+11  2.248220e+11   
         2022  3.888186e+11     3.912218e+11  5.545520e+11  1.882430e+11   
TSLA.US  2020  7.761015e+11     7.582119e+11  3.153600e+10  6.900000e+08   
         2021  7.761015e+11     7.582119e+11  5.382300e+10  5.519000e+09   
         2022  7.761015e+11     7.582119e+11  8.146200e+10  1.258300e+10   

                     ebitda  revGrowth  ebitdaMargin  profitMargin  
ticker   year                                                       
AAPL.US  2020  7.734400e+10   0.055121      0.281748      0.209136  
         2021  1.202330e+11   0.332594      0.328670      0.258818  
         2022  1.305410e+11   0.077938      0.331047      0.253096  
AMZN.US  2020  4.807900e+10   0.376234      0.124536      0.055252  
         2021  5.931200e+10   0.216954      0.126244      0.071014  
         2022  5.416900e+10   0.093995      0.105391     -0.005296  
F.US     2020  1.627500e+10  -0.184452      0.128004     -0.010036  
         2021  2.333800e+10   0.072335      0.171174      0.131560  
         2022  1.889400e+10   0.159277      0.119539     -0.013615  
GM.US    2020  2.174300e+10  -0.107493      0.177516      0.052472  
         2021  2.571700e+10   0.036894      0.202490      0.078887  
         2022  2.387400e+10   0.234095      0.152321      0.063381  
GOOGL.US 2020  5.492100e+10   0.127705      0.300892      0.220619  
         2021  9.115500e+10   0.411501      0.353812      0.295117  
         2022  9.077000e+10   0.097808      0.320928      0.212038  
INTC.US  2020  3.801900e+10   0.082012      0.488256      0.268394  
         2021  3.650000e+10   0.014859      0.461885      0.251417  
         2022  1.110500e+10  -0.202091      0.176119      0.127097  
TCEHY.US 2020  1.841880e+11   0.277705      0.382082      0.331589  
         2021  1.662470e+11   0.161916      0.296807      0.401383  
         2022  1.491310e+11  -0.009937      0.268922      0.339451  
TSLA.US  2020  4.224000e+09   0.283099      0.133942      0.021880  
         2021  9.598000e+09   0.706716      0.178325      0.102540  
         2022  1.783300e+10   0.513517      0.218912      0.154465

In [11]:
### Trading Multiple
df_all["EVofRevenue"] = df_all["enterpriseValue"] / df_all["totalRevenue"]
df_all["EVofEbitda"] = df_all["enterpriseValue"] / df_all["ebitda"]
df_all["PofE"] = df_all["marketCap"] / df_all["netIncome"]
df_all

marketCap  enterpriseValue  totalRevenue     netIncome  \
ticker   year                                                              
AAPL.US  2020  2.945234e+12     2.994758e+12  2.745150e+11  5.741100e+10   
         2021  2.945234e+12     2.994758e+12  3.658170e+11  9.468000e+10   
         2022  2.945234e+12     2.994758e+12  3.943280e+11  9.980300e+10   
AMZN.US  2020  1.519408e+12     1.592233e+12  3.860640e+11  2.133100e+10   
         2021  1.519408e+12     1.592233e+12  4.698220e+11  3.336400e+10   
         2022  1.519408e+12     1.592233e+12  5.139830e+11 -2.722000e+09   
F.US     2020  4.239124e+10     1.458183e+11  1.271440e+11 -1.276000e+09   
         2021  4.239124e+10     1.458183e+11  1.363410e+11  1.793700e+10   
         2022  4.239124e+10     1.458183e+11  1.580570e+11 -2.152000e+09   
GM.US    2020  3.956428e+10     1.235623e+11  1.224850e+11  6.427000e+09   
         2021  3.956428e+10     1.235623e+11  1.270040e+11  1.001900e+10   
         2022  3.956428e+10     1.235623e+11  1.567350e+11  9.934000e+09   
GOOGL.US 2020  1.697607e+12     1.606718e+12  1.825270e+11  4.026900e+10   
         2021  1.697607e+12     1.606718e+12  2.576370e+11  7.603300e+10   
         2022  1.697607e+12     1.606718e+12  2.828360e+11  5.997200e+10   
INTC.US  2020  1.894670e+11     2.103227e+11  7.786700e+10  2.089900e+10   
         2021  1.894670e+11     2.103227e+11  7.902400e+10  1.986800e+10   
         2022  1.894670e+11     2.103227e+11  6.305400e+10  8.014000e+09   
TCEHY.US 2020  3.888186e+11     3.912218e+11  4.820640e+11  1.598470e+11   
         2021  3.888186e+11     3.912218e+11  5.601180e+11  2.248220e+11   
         2022  3.888186e+11     3.912218e+11  5.545520e+11  1.882430e+11   
TSLA.US  2020  7.761015e+11     7.582119e+11  3.153600e+10  6.900000e+08   
         2021  7.761015e+11     7.582119e+11  5.382300e+10  5.519000e+09   
         2022  7.761015e+11     7.582119e+11  8.146200e+10  1.258300e+10   

                     ebitda  revGrowth  ebitdaMargin  profitMargin  \
ticker   year                                                        
AAPL.US  2020  7.734400e+10   0.055121      0.281748      0.209136   
         2021  1.202330e+11   0.332594      0.328670      0.258818   
         2022  1.305410e+11   0.077938      0.331047      0.253096   
AMZN.US  2020  4.807900e+10   0.376234      0.124536      0.055252   
         2021  5.931200e+10   0.216954      0.126244      0.071014   
         2022  5.416900e+10   0.093995      0.105391     -0.005296   
F.US     2020  1.627500e+10  -0.184452      0.128004     -0.010036   
         2021  2.333800e+10   0.072335      0.171174      0.131560   
         2022  1.889400e+10   0.159277      0.119539     -0.013615   
GM.US    2020  2.174300e+10  -0.107493      0.177516      0.052472   
         2021  2.571700e+10   0.036894      0.202490      0.078887   
         2022  2.387400e+10   0.234095      0.152321      0.063381   
GOOGL.US 2020  5.492100e+10   0.127705      0.300892      0.220619   
         2021  9.115500e+10   0.411501      0.353812      0.295117   
         2022  9.077000e+10   0.097808      0.320928      0.212038   
INTC.US  2020  3.801900e+10   0.082012      0.488256      0.268394   
         2021  3.650000e+10   0.014859      0.461885      0.251417   
         2022  1.110500e+10  -0.202091      0.176119      0.127097   
TCEHY.US 2020  1.841880e+11   0.277705      0.382082      0.331589   
         2021  1.662470e+11   0.161916      0.296807      0.401383   
         2022  1.491310e+11  -0.009937      0.268922      0.339451   
TSLA.US  2020  4.224000e+09   0.283099      0.133942      0.021880   
         2021  9.598000e+09   0.706716      0.178325      0.102540   
         2022  1.783300e+10   0.513517      0.218912      0.154465   

               EVofRevenue  EVofEbitda         PofE  
ticker   year                                        
AAPL.US  2020    10.909268   38.719974    51.300861  
         2021     8.186491   24.907951    31.107242  
    

In [12]:
df_comp = df_all[["totalRevenue", "ebitda", "netIncome", "revGrowth", "ebitdaMargin", "profitMargin", "EVofRevenue", "EVofEbitda", "PofE"]].unstack(level='year')
df_comp.loc["Minimum", :] = df_comp.min()
df_comp.loc["Mean", :] = df_comp.mean()
df_comp.loc[" Median", :] = df_comp.median()
df_comp.loc["Maximum", :] = df_comp.max()
df_comp.loc[:, "revGrowth":]

revGrowth                     ebitdaMargin                      \
year          2020      2021      2022         2020      2021      2022   
ticker                                                                    
AAPL.US   0.055121  0.332594  0.077938     0.281748  0.328670  0.331047   
AMZN.US   0.376234  0.216954  0.093995     0.124536  0.126244  0.105391   
F.US     -0.184452  0.072335  0.159277     0.128004  0.171174  0.119539   
GM.US    -0.107493  0.036894  0.234095     0.177516  0.202490  0.152321   
GOOGL.US  0.127705  0.411501  0.097808     0.300892  0.353812  0.320928   
INTC.US   0.082012  0.014859 -0.202091     0.488256  0.461885  0.176119   
TCEHY.US  0.277705  0.161916 -0.009937     0.382082  0.296807  0.268922   
TSLA.US   0.283099  0.706716  0.513517     0.133942  0.178325  0.218912   
Minimum  -0.184452  0.014859 -0.202091     0.124536  0.126244  0.105391   
Mean      0.080609  0.218736  0.084724     0.237946  0.249517  0.199841   
 Median   0.081310  0.189435  0.089359     0.207731  0.226003  0.187980   
Maximum   0.376234  0.706716  0.513517     0.488256  0.461885  0.331047   

         profitMargin                     EVofRevenue                       \
year             2020      2021      2022        2020       2021      2022   
ticker                                                                       
AAPL.US      0.209136  0.258818  0.253096   10.909268   8.186491  7.594585   
AMZN.US      0.055252  0.071014 -0.005296    4.124271   3.389012  3.097831   
F.US        -0.010036  0.131560 -0.013615    1.146875   1.069512  0.922568   
GM.US        0.052472  0.078887  0.063381    1.008795   0.972901  0.788352   
GOOGL.US     0.220619  0.295117  0.212038    8.802632   6.236364  5.680741   
INTC.US      0.268394  0.251417  0.127097    2.701050   2.661504  3.335596   
TCEHY.US     0.331589  0.401383  0.339451    0.811556   0.698463  0.705474   
TSLA.US      0.021880  0.102540  0.154465   24.042741  14.087135  9.307553   
Minimum     -0.010036  0.071014 -0.013615    0.811556   0.698463  0.705474   
Mean         0.126586  0.184639  0.124111    6.039860   4.222205  3.570908   
 Median      0.090919  0.158099  0.125604    3.412661   3.025258  3.216714   
Maximum      0.331589  0.401383  0.339451   24.042741  14.087135  9.307553   

          EVofEbitda                               PofE              \
year            2020       2021       2022         2020        2021   
ticker                                                                
AAPL.US    38.719974  24.907951  22.941127    51.300861   31.107242   
AMZN.US    33.117006  26.845032  29.393796    71.230040   45.540343   
F.US        8.959649   6.248106   7.717703   -33.221975    2.363341   
GM.US       5.682855   4.804694   5.175602     6.155948    3.948925   
GOOGL.US   29.255077  17.626220  17.700981    42.156679   22.327244   
INTC.US     5.532041   5.762265  18.939458     9.065842    9.536291   
TCEHY.US    2.124035   2.353256   2.623343     2.432442    1.729451   
TSLA.US   179.500916  78.996861  42.517348  1124.784775  140.623572   
Minimum     2.124035   2.353256   2.623343   -33.221975    1.729451   
Mean       33.890621  18.877516  16.625856   137.853626   28.767318   
 Median    19.107363  11.937163  17.163419    25.611261   15.931767   
Maximum   179.500916  78.996861  42.517348  1124.784775  140.623572   

                      
year            2022  
ticker                
AAPL.US    29.510473  
AMZN.US  -558.195440  
F.US      -19.698532  
GM.US       3.982713  
GOOGL.US   28.306665  
INTC.US    23.642006  
TCEHY.US    2.065514  
TSLA.US    61.678574  
Minimum  -558.195440  
Mean     -109.655941  
 Median     3.024114  
Maximum    61.678574

In [13]:
year = "2022"
df_comp.columns = df_comp.columns.map('_'.join)
df_comp = df_comp.loc[:, df_comp.columns.str.contains(year)]
df_comp.columns = [i[:-5] for i in df_comp.columns]
df_comp

,totalRevenue,ebitda,netIncome,revGrowth,ebitdaMargin,profitMargin,EVofRevenue,EVofEbitda,PofE
ticker,,,,,,,,,
AAPL.US,3.943280e+11,1.305410e+11,9.980300e+10,0.077938,0.331047,0.253096,7.594585,22.941127,29.510473
AMZN.US,5.139830e+11,5.416900e+10,-2.722000e+09,0.093995,0.105391,-0.005296,3.097831,29.393796,-558.195440
F.US,1.580570e+11,1.889400e+10,-2.152000e+09,0.159277,0.119539,-0.013615,0.922568,7.717703,-19.698532
GM.US,1.567350e+11,2.387400e+10,9.934000e+09,0.234095,0.152321,0.063381,0.788352,5.175602,3.982713
GOOGL.US,2.828360e+11,9.077000e+10,5.997200e+10,0.097808,0.320928,0.212038,5.680741,17.700981,28.306665
INTC.US,6.305400e+10,1.110500e+10,8.014000e+09,-0.202091,0.176119,0.127097,3.335596,18.939458,23.642006
TCEHY.US,5.545520e+11,1.491310e+11,1.882430e+11,-0.009937,0.268922,0.339451,0.705474,2.623343,2.065514
TSLA.US,8.146200e+10,1.783300e+10,1.258300e+10,0.513517,0.218912,0.154465,9.307553,42.517348,61.678574
Minimum,6.305400e+10,1.110500e+10,-2.722000e+09,-0.202091,0.105391,-0.013615,0.705474,2.623343,-558.195440


In [14]:
outshares = client.get_fundamental_equity(stock, filter_='SharesStats::SharesOutstanding')
df_price = df_comp.loc["Minimum":, "EVofRevenue":]
# Financial metric
df_price[stock[:-3]+"_revenue"] = df_comp.loc[stock, "totalRevenue"]
df_price[stock[:-3]+"_ebitda"] = df_comp.loc[stock, "ebitda"]
df_price["EPS"] = df_comp.loc[stock, "netIncome"] / outshares
# Enterprise Value
df_price["EV_revenue"] = df_price[stock[:-3]+"_revenue"] *df_price["EVofRevenue"]
df_price["EV_ebitda"] = df_price[stock[:-3]+"_ebitda"] *df_price["EVofEbitda"]

bs = client.get_fundamental_equity(stock, filter_='Financials::Balance_Sheet::yearly')
df_price["cash"] = float(pd.DataFrame(bs).loc["cash"].values[0])
df_price["debt"] = float(pd.DataFrame(bs).loc["totalLiab"].values[0])
df_price["outshares"] = outshares

In [15]:
df_price.T

ticker,Minimum,Mean,Median,Maximum
EVofRevenue,7.054736e-01,3.570908e+00,3.216714e+00,9.307553e+00
EVofEbitda,2.623343e+00,1.662586e+01,1.716342e+01,4.251735e+01
PofE,-5.581954e+02,-1.096559e+02,3.024114e+00,6.167857e+01
TSLA_revenue,8.146200e+10,8.146200e+10,8.146200e+10,8.146200e+10
TSLA_ebitda,1.783300e+10,1.783300e+10,1.783300e+10,1.783300e+10
EPS,3.958263e+00,3.958263e+00,3.958263e+00,3.958263e+00
EV_revenue,5.746929e+10,2.908933e+11,2.620399e+11,7.582119e+11
EV_ebitda,4.678208e+10,2.964889e+11,3.060752e+11,7.582119e+11
cash,1.625300e+10,1.625300e+10,1.625300e+10,1.625300e+10
debt,3.644000e+10,3.644000e+10,3.644000e+10,3.644000e+10


In [101]:
df_price["EQ_revenue"] = df_price["EV_revenue"] +  df_price["cash"] - df_price["debt"]
df_price["EQ_ebitda"] = df_price["EV_ebitda"] +  df_price["cash"] - df_price["debt"]
df_price["EQ_netIncome"] = df_price["PofE"] *  df_price["EPS"] * df_price["outshares"]

df_price["ISP_revenue"] = df_price["EQ_revenue"] / df_price["outshares"]
df_price["ISP_ebitda"] = df_price["EQ_ebitda"] / df_price["outshares"]
df_price["ISP_netIncome"] = df_price["EQ_netIncome"] / df_price["outshares"]

In [102]:
df_price.T

ticker,Minimum,Mean,Median,Maximum
EVofRevenue,8.414636e-01,3.604595e+00,3.273753e+00,9.005656e+00
EVofEbitda,3.129030e+00,1.682282e+01,1.731754e+01,4.113827e+01
PofE,-5.608530e+02,-1.104447e+02,2.993866e+00,5.964233e+01
TSLA_revenue,8.146200e+10,8.146200e+10,8.146200e+10,8.146200e+10
TSLA_ebitda,1.783300e+10,1.783300e+10,1.783300e+10,1.783300e+10
EPS,3.958263e+00,3.958263e+00,3.958263e+00,3.958263e+00
EV_revenue,6.854731e+10,2.936375e+11,2.666864e+11,7.336188e+11
EV_ebitda,5.579999e+10,3.000014e+11,3.088237e+11,7.336188e+11
cash,1.625300e+10,1.625300e+10,1.625300e+10,1.625300e+10
debt,3.644000e+10,3.644000e+10,3.644000e+10,3.644000e+10
